In [ ]:
from ddm.data_handling import read_file
from ddm.processing import ddm_numpy, radial_profile #need to make sure we use the proper ddm function with dask functionality
from ddm.fitting import findMeanSqFFT_np, computeAB
from ddm.plotting import plotSingleExpFit, plotISF
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
filename = '../data/test_dataset_drift.tif'

ddmData = read_file(filename, delayed=False)

In [ ]:
ddmData = ddmData[0:500,:,:]

### Default algorithm

In [ ]:
%%time
intTaus = np.arange(1,len(ddmData)//2)
# ddmMatrix = np.array([None for j in range(len(intTaus))])
ddmMatrix = []
for tau in tqdm(intTaus):
    # ddmMatrix[tau] = ddm_numpy(ddmData, tau) #at this point we have the 0th axis as taus and the 1st axis as qs
    ddmMatrix.append(ddm_numpy(ddmData, tau))
ddmMatrix = np.asarray(ddmMatrix)

In [ ]:
plt.plot(ddmMatrix[10,:])
plt.plot(ddmMatrix[25,:])
plt.plot(ddmMatrix[48,:])
plt.xlim(0, 80)
plt.show()

### Faster algorithm with numpy

In [ ]:
%%time
intTaus = np.arange(1,len(ddmData)//2)
num_frames, height, width = ddmData.shape
ddmMatrix_fast = []

img_fft = np.fft.fft2(ddmData).astype("complex64")
for tau in tqdm(intTaus):
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = np.abs(img_diff) ** 2
    img_sum = np.sum(img_fft_sq, axis=0)
    fft_shift = np.fft.fftshift(img_sum)
    gTau = fft_shift / (num_frames - tau)
    gTauRadial = radial_profile(gTau, (width / 2.0, height / 2.0))
    ddmMatrix_fast.append(gTauRadial)
ddmMatrix_fast = np.asarray(ddmMatrix_fast)


In [ ]:
plt.plot(ddmMatrix_fast[10,:])
plt.plot(ddmMatrix_fast[25,:])
plt.plot(ddmMatrix_fast[48,:])
plt.xlim(0, 80)
plt.show()

In [ ]:
np.testing.assert_allclose(ddmMatrix, ddmMatrix_fast, rtol=1e-3)

### Optimize with dask delayed

In [ ]:
def calc_matrix(img_fft, tau, num_frames, height, width):
    img_diff = img_fft[:-tau, :, :] - img_fft[tau:, :, :]
    img_fft_sq = np.abs(img_diff) ** 2
    img_sum = np.sum(img_fft_sq, axis=0)
    fft_shift = np.fft.fftshift(img_sum)
    gTau = fft_shift / (num_frames - tau)
    gTauRadial = radial_profile(gTau, (width / 2.0, height / 2.0))
    return gTauRadial

In [ ]:
import dask
from dask.diagnostics import ProgressBar, ResourceProfiler

In [ ]:
%%time
taus = np.arange(1,len(ddmData)//2)
num_frames, height, width = ddmData.shape
results = []

img_fft = np.fft.fft2(ddmData).astype("complex64")
for tau in taus:
    result = dask.delayed(calc_matrix)(img_fft, tau, num_frames, height, width)
    results.append(result)

with ProgressBar() as prof, ResourceProfiler() as rprof:
    out = dask.compute(*results)
out = np.asarray(out)

In [ ]:
rprof.results

In [ ]:
del img_fft

In [ ]:
for row in out:
    plt.plot(row, color='k', alpha=0.1)
    plt.xlim(0, 100)
plt.show()

### Store result

In [ ]:
file_out = "../data/test_dataset_drift_ddm.txt"
np.savetxt(file_out, out)